In [0]:
import pyspark

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
storageAccountKey = 'tyxSGF4/AI6Hma1zFD6zT4tiIvbKdP7tvE7lmaEOciWjabp+jPuadpim2/II0RNcno75CiETVzDp+ASt/JfHJQ=='
spark.conf.set('fs.azure.account.key.bayadapoc.dfs.core.windows.net', storageAccountKey)

In [0]:
df = spark.read.format('csv').option('header', True).load("abfss://trainingexternal@bayadapoc.dfs.core.windows.net/data/PW_MW_DR_01012023.csv")
display(df)

In [0]:
df.printSchema()

In [0]:
df2 = df.select('DATE_OF_PRICING', 'ROW_ID', 'STATE_NAME', 'MARKET_NAME', 'PRODUCTGROUP_NAME', 'PRODUCT_NAME', 'VARIETY', 'ORIGIN', 'ARRIVAL_IN_TONNES')
df2.display()

In [0]:
df.count()

In [0]:
df2.select('row_id').distinct().count()

In [0]:
from pyspark.sql.functions import *
whensumnullcount = df.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns])
whensumnullcount.display()

**Filter ARRIVAL_IN_TONNES with grater than 20 and State should be Uttar Pradesh , West Bengal, Jharkhand, Gujarat**

In [0]:
filterdf = df.filter((df['ARRIVAL_IN_TONNES'] > 20) & (df['STATE_NAME'].isin(['Uttar Pradesh ', 'West Bengal ', 'Jharkhand ', 'Gujarat '])))
filterdf.count()

In [0]:
filterdf.display(50)

**convert DATE_OF_PRICING to 'yyyyMMDD' format.**

In [0]:
# convertdf = df.withColumn('DATE_OF_PRICING', to_date(df.DATE_OF_PRICING))
convertdf2 = df.withColumn('DATE_OF_PRICING_updated', date_format(to_date(df.DATE_OF_PRICING, 'MM/dd/yyyy'), 'yyyyMMDD'))
convertdf2.display()
convertdf2.printSchema()

**crete new column year and fetch year from DATE_OF_PRICING.**

In [0]:
yeardf = df.withColumn('year', year(to_date(df.DATE_OF_PRICING, 'MM/dd/yyyy')))
yeardf.display()

**Find which state has more market sort them by descending.**

In [0]:
statedf = df.groupBy('STATE_NAME').agg(count('MARKET_NAME').alias('count_of_markets'))
# display(statedf)

#sort it by descending
display(statedf.sort(desc('count_of_markets')))

display(statedf.first())
display(statedf.select(max(statedf.count_of_markets).first()))

**Add new column to convert tons in to kg for column ARRIVAL_IN_TONNES.**

In [0]:
KGdf = df.withColumn('ARRIVAL_IN_KG', (df.ARRIVAL_IN_TONNES).cast('int') * 1000)
KGdf.display()
KGdf.printSchema()

**Create another dataframe which consist unique state and market name . save this into table under your schema name.**

In [0]:
newdf = df.select('STATE_NAME', 'MARKET_NAME').distinct()
newdf.write.mode('overwrite').saveAsTable('training.piyush.Unique_State_Market_names')

**Create dataframe for unique product related details.(Not price) Save that into table under your schema**

In [0]:
display(df.select('PRODUCT_NAME').distinct())